In [1]:
import numpy as np
from src.SERGIO.SERGIO.sergio import sergio
import pandas as pd
from src.algs import ges_bic_tetrad_local_learn, pc_fisherz_tetrad_local_learn, pc_kci_local_learn, direct_lingam_tetrad_local_learn, dag_gnn_local_learn, genie3_local_learn, dagma_nonlinear_local_learn

/homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-30 16:49:41,522 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2026-01-30 16:49:41,632 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


In [2]:
sim = sergio(number_genes=3,
             number_bins=3,
             number_sc=1000,
             noise_params=0.2,
             decays=0.8,
             sampling_state=15,
             noise_type='dpd',
             dynamics=False)

In [ ]:
# build a regulatory network and assign random hill coefficients
# g0 -> g1 -| g2
# target gene id, number of target’s regulators, regulator ID_1,…, regulator ID_n, K_1,…,K_n, hill_coeff_1, …, hill_coeff_n
data = [[1, 1, 0, 2.5, 2], [2, 1, 1, -1.3, 2]]
df = pd.DataFrame(data=data)
df.to_csv("./data/sergio_input_file_targets.csv", header=False, index=False)

# Example: input_file_regs, for GRN g0 --> g1 --| g2, in three cell types:
data = [0, 0.5, 1.5, 3]
#data = [0,3] # 1 cell type
df = pd.DataFrame(data=data)
df.T.to_csv("./data/sergio_input_file_regs.csv", header=False, index=False)

In [4]:
sim.build_graph(input_file_taregts="./data/sergio_input_file_targets.csv", input_file_regs="./data/sergio_input_file_regs.csv")
sim.graph_

{0: {'targets': [1], 'rates': [0.5, 1.5, 3.0], 'regs': [], 'level': 2},
 1: {'targets': [2], 'params': [(0, 2.5, 2.0, 0)], 'regs': [0], 'level': 1},
 2: {'targets': [], 'params': [(1, -1.3, 2.0, 0)], 'regs': [1], 'level': 0}}

In [5]:
sim.simulate()

Start simulating new level
There are 1 genes to simulate in this layer
Done with current level
Start simulating new level
There are 1 genes to simulate in this layer
Done with current level
Start simulating new level
There are 1 genes to simulate in this layer
Done with current level


In [6]:
expr = sim.getExpressions()
expr = np.concatenate(expr, axis = 1)
df_expr = pd.DataFrame(data=expr).T
df_expr.shape

(3000, 3)

In [7]:
ges_bic_tetrad_local_learn(df_expr)

Jan 30, 2026 4:49:45 PM java.util.prefs.FileSystemPreferences$6 run


array([[0, 3, 3],
       [3, 0, 3],
       [3, 3, 0]])

In [8]:
pc_fisherz_tetrad_local_learn(df_expr)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


array([[0, 3, 3],
       [3, 0, 3],
       [3, 3, 0]])

In [9]:
g = pc_kci_local_learn(df_expr)
g.to_nx_graph()
g.nx_graph.edges()

Depth=1, working on node 2: 100%|██████████| 3/3 [00:32<00:00, 10.82s/it]  


OutEdgeView([(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)])

In [10]:
dag_gnn_local_learn(df_expr, seed=0)

2026-01-30 16:50:21,700 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:165] - INFO: GPU is available.


cuda


2026-01-30 16:50:30,361 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 0, epoch: 49, h_new: 0.03487601038063426
2026-01-30 16:50:43,523 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 1, epoch: 49, h_new: 0.005584981639270126
2026-01-30 16:50:56,698 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 2, epoch: 49, h_new: 0.0008105260183164376
2026-01-30 16:51:09,980 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 3, epoch: 49, h_new: 0.00017556777500260523
2026-01-30 16:51:30,092 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253]

Tensor([[ 0.   ,  0.   ,  0.   ],
        [ 1.521,  0.   ,  0.   ],
        [ 0.   , -1.394,  0.   ]])

In [11]:
direct_lingam_tetrad_local_learn(df_expr)

Graph Nodes:
0;1;2

Graph Edges:
1. 0 --> 1
2. 2 --> 0
3. 2 --> 1




array([[0, 2, 3],
       [3, 0, 3],
       [2, 2, 0]])

In [12]:
dagma_nonlinear_local_learn(df_expr)

  0%|          | 24/230000.0 [00:00<1:11:30, 53.60it/s]


array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])